In [1]:
import numpy as np
import os
import shutil
from pImpactR import MLI as mli
from pImpactR import opt
from pImpactR.util import Me
from copy import deepcopy as copy
import time
import pickle

# Read MLI input file

In [2]:
elems0,lattices,labor = mli.readInputfile('mli.in.t3_iota_8_4_t0p4')

setarclen is not recognized. skipping...
autoconcat is not recognized. skipping...
autoapply is not recognized. skipping...
reftraj is not recognized. skipping...
fit is not recognized. skipping...
bip is not recognized. skipping...
tip is not recognized. skipping...


In [3]:
energy=elems0[0].energy*1.0e9
gam0 = energy/Me
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [4]:
elems = []
for item in elems0:
    if not item.name in ['tasm','aim','vary','clear','anaprint']:
        elems.append(item)

In [5]:
labor = ['iotaline','mapout','fin']

# thick sext 2 thin multipole

In [6]:
elemList,latticeList=mli.sext2thin(elems,lattices,brho=0.50204778184582999)

In [7]:
mli.writeInputfile(elemList,latticeList,labor)

In [8]:
mli.run()

In [9]:
M,G=mli.readTransferMap()

In [10]:
M

,1,2,3,4,5,6
1,0.999968,0.000070,0.000000,0.000000e+00,0.0,-1.147949e-06
2,-1.454445,0.999930,0.000000,0.000000e+00,0.0,-2.587771e-07
3,0.000000,0.000000,0.999995,3.984744e-07,0.0,0.000000e+00
4,0.000000,0.000000,-1.454438,1.000004e+00,0.0,0.000000e+00
5,0.000002,-0.000001,0.000000,0.000000e+00,1.0,-1.297155e+00
6,0.000000,0.000000,0.000000,0.000000e+00,0.0,1.000000e+00


In [11]:
G.head()

,exponents,GP
28,f( 30 00 00 ),-1.254486
29,f( 21 00 00 ),-2.937953
33,f( 20 00 01 ),0.971029
34,f( 12 00 00 ),-2.099518
38,f( 11 00 01 ),0.513714


In [12]:
indexThin = []
k2lList = []
for i,item in enumerate(elemList):
    if 'thlm' == item.elem:
        indexThin.append(i)
        k2lList.append(item.k2l)
nThin = len(indexThin)
print(nThin)

18


# Optimize

In [13]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx
print(k)

1.4544633270832747


In [14]:
G2_ref = G.loc[28:76].copy()
G2_ref['GP'] = 0
G2_ref.loc[33,'GP'] = 0.5*k/bet0
G2_ref.loc[67,'GP'] = 0.5*k/bet0
G2_ref.head()

,exponents,GP
28,f( 30 00 00 ),0.000000
29,f( 21 00 00 ),0.000000
33,f( 20 00 01 ),0.727236
34,f( 12 00 00 ),0.000000
38,f( 11 00 01 ),0.000000


In [15]:
G3_ref = G.loc[84:200].copy()
G3_ref['GP'] = 0
G3_ref.loc[104,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.loc[184,'GP'] = k/(2.0*bet0*gam0)**2

In [16]:
G4_ref = G.loc[210:450].copy()
G4_ref['GP'] = 0

In [17]:
G5_ref = G.loc[462:910].copy()
G5_ref['GP'] = 0

In [18]:
print(len(G2_ref),len(G3_ref),len(G4_ref),len(G5_ref))

18 37 65 109


In [19]:
#%%
def objFunc(arg): 
    target = opt.id_generator()  # generage random directory name
    while os.path.exists(target):  
        target = opt.id_generator()
    shutil.copytree('origin', target)
#     time.sleep(0.1)
    os.chdir(target) # cd to the randome directory and
    
    for i,j in enumerate(indexThin):
        elemList[j]['k2l']=arg[i]
    for i,j in enumerate(indexThin):
        elemList[j]['k3l']=arg[i+nThin]
        
    mli.writeInputfile(elemList,latticeList,labor)
    mli.run()
#     time.sleep(0.1)
    M,G = mli.readTransferMap()
    
    obj = 1.0e30* np.sum((G.loc[28 :76 ,'GP'].values-G2_ref['GP'].values)**2) \
         +1.0e22* np.sum((G.loc[84 :200,'GP'].values-G3_ref['GP'].values)**2) \
         +1.0e12* np.sum((G.loc[210:450,'GP'].values-G4_ref['GP'].values)**2) \
         +        np.sum((G.loc[462:910,'GP'].values-G5_ref['GP'].values)**2)
#     time.sleep(0.1)
    os.chdir('..')
#     time.sleep(0.1)
    shutil.rmtree(target)
#     time.sleep(0.1)
    return obj

In [20]:
objFunc(k2lList+[0.0]*nThin)

1.0810442738177038e+32

In [21]:
#%% run optim
bounds = [(-100,100)]*nThin + [(-10000,10000)]*nThin
# result=opt.differential_evolution(objFunc, bounds, ncore=32, popsize=32*4,
#                                   disp=True, polish=True, maxtime=60*10) 
                                    # stop running at maximum 10 min

In [22]:
# with open('result.thin.sext.oct','wb') as fp:
#     pickle.dump(result,fp)

In [23]:
with open('result.thin.sext.oct','rb') as fp:
    result=pickle.load(fp)
result.population_energies[0]

3.540488650295599e+32

In [24]:
while True:
    previous_result = result
    if hasattr(result,'x'): 
        break
    result = opt.differential_evolution(objFunc, bounds, ncore=32, 
                                           prev_result=previous_result, 
                                           disp=True, polish=True, maxtime=60*10)
    with open('result.thin.sext.oct','wb') as fp:
        pickle.dump(result,fp)
#         time.sleep(0.1)

differential_evolution step 0: f(x)= 3.54049e+32
differential_evolution step 0: f(x)= 3.54049e+32
differential_evolution step 1: f(x)= 3.54049e+32
differential_evolution step 2: f(x)= 3.54049e+32
differential_evolution step 3: f(x)= 3.54049e+32
differential_evolution step 4: f(x)= 3.54049e+32
differential_evolution step 5: f(x)= 3.54049e+32
differential_evolution step 6: f(x)= 3.54049e+32
differential_evolution step 7: f(x)= 3.54049e+32
differential_evolution step 8: f(x)= 3.54049e+32
differential_evolution step 9: f(x)= 3.54049e+32
differential_evolution step 10: f(x)= 3.54049e+32
differential_evolution step 11: f(x)= 3.54049e+32
differential_evolution step 12: f(x)= 3.54049e+32
differential_evolution step 13: f(x)= 3.54049e+32
differential_evolution step 14: f(x)= 3.54049e+32
differential_evolution step 15: f(x)= 3.54049e+32
differential_evolution step 16: f(x)= 3.54049e+32
differential_evolution step 17: f(x)= 3.54049e+32
differential_evolution step 18: f(x)= 3.54049e+32
differentia

differential_evolution step 13: f(x)= 3.0172e+32
differential_evolution step 14: f(x)= 3.0172e+32
differential_evolution step 15: f(x)= 3.0172e+32
differential_evolution step 16: f(x)= 3.0172e+32
differential_evolution step 17: f(x)= 3.0172e+32
differential_evolution step 18: f(x)= 3.0172e+32
differential_evolution step 19: f(x)= 2.79726e+32
differential_evolution step 20: f(x)= 2.79726e+32
differential_evolution step 21: f(x)= 2.79726e+32
differential_evolution step 22: f(x)= 2.79726e+32
differential_evolution step 23: f(x)= 2.79726e+32
differential_evolution step 24: f(x)= 2.79726e+32
differential_evolution step 25: f(x)= 2.79726e+32
differential_evolution step 26: f(x)= 2.79726e+32
differential_evolution step 27: f(x)= 2.79726e+32
differential_evolution step 28: f(x)= 2.00914e+32
differential_evolution step 29: f(x)= 2.00914e+32
differential_evolution step 30: f(x)= 2.00914e+32
differential_evolution step 31: f(x)= 2.00914e+32
differential_evolution step 32: f(x)= 2.00914e+32
differ

differential_evolution step 44: f(x)= 1.71151e+32
differential_evolution step 45: f(x)= 1.71151e+32
differential_evolution step 46: f(x)= 1.71151e+32
differential_evolution step 47: f(x)= 1.71151e+32
differential_evolution step 48: f(x)= 1.71151e+32
differential_evolution step 49: f(x)= 1.71151e+32
differential_evolution step 0: f(x)= 1.71151e+32
differential_evolution step 0: f(x)= 1.71151e+32
differential_evolution step 1: f(x)= 1.71151e+32
differential_evolution step 2: f(x)= 1.71151e+32
differential_evolution step 3: f(x)= 1.71151e+32
differential_evolution step 4: f(x)= 1.71151e+32
differential_evolution step 5: f(x)= 1.71151e+32
differential_evolution step 6: f(x)= 1.71151e+32
differential_evolution step 7: f(x)= 1.71151e+32
differential_evolution step 8: f(x)= 1.71151e+32
differential_evolution step 9: f(x)= 1.71151e+32
differential_evolution step 10: f(x)= 1.71151e+32
differential_evolution step 11: f(x)= 1.71151e+32
differential_evolution step 12: f(x)= 1.71151e+32
differentia

differential_evolution step 15: f(x)= 1.17366e+32
differential_evolution step 16: f(x)= 1.17366e+32
differential_evolution step 17: f(x)= 1.17366e+32
differential_evolution step 18: f(x)= 1.17366e+32
differential_evolution step 19: f(x)= 1.17366e+32
differential_evolution step 20: f(x)= 1.17366e+32
differential_evolution step 21: f(x)= 1.17366e+32
differential_evolution step 22: f(x)= 1.17366e+32
differential_evolution step 23: f(x)= 1.17366e+32
differential_evolution step 24: f(x)= 1.17366e+32
differential_evolution step 25: f(x)= 1.17366e+32
differential_evolution step 26: f(x)= 1.17366e+32
differential_evolution step 27: f(x)= 1.17366e+32
differential_evolution step 28: f(x)= 1.17366e+32
differential_evolution step 29: f(x)= 1.17366e+32
differential_evolution step 30: f(x)= 1.17366e+32
differential_evolution step 31: f(x)= 1.17366e+32
differential_evolution step 32: f(x)= 1.17366e+32
differential_evolution step 33: f(x)= 1.17366e+32
differential_evolution step 34: f(x)= 1.17366e+32


differential_evolution step 49: f(x)= 1.14257e+32
differential_evolution step 0: f(x)= 1.14257e+32
differential_evolution step 0: f(x)= 1.14257e+32
differential_evolution step 1: f(x)= 1.14257e+32
differential_evolution step 2: f(x)= 1.14257e+32
differential_evolution step 3: f(x)= 1.14257e+32
differential_evolution step 4: f(x)= 1.14257e+32
differential_evolution step 5: f(x)= 1.14257e+32
differential_evolution step 6: f(x)= 1.14257e+32
differential_evolution step 7: f(x)= 1.14257e+32
differential_evolution step 8: f(x)= 1.14257e+32
differential_evolution step 9: f(x)= 1.14257e+32
differential_evolution step 10: f(x)= 1.14257e+32
differential_evolution step 11: f(x)= 1.14257e+32
differential_evolution step 12: f(x)= 1.14257e+32
differential_evolution step 13: f(x)= 1.08356e+32
differential_evolution step 14: f(x)= 1.08356e+32
differential_evolution step 15: f(x)= 1.08356e+32
differential_evolution step 16: f(x)= 1.08356e+32
differential_evolution step 17: f(x)= 1.08356e+32
differentia

...

differential_evolution step 42: f(x)= 8.06207e+30
differential_evolution step 43: f(x)= 8.06207e+30
differential_evolution step 44: f(x)= 8.06207e+30
differential_evolution step 45: f(x)= 8.06207e+30
differential_evolution step 46: f(x)= 8.06207e+30
differential_evolution step 47: f(x)= 8.06207e+30
differential_evolution step 48: f(x)= 8.06207e+30
differential_evolution step 49: f(x)= 8.06207e+30
differential_evolution step 0: f(x)= 8.06207e+30
differential_evolution step 0: f(x)= 8.06207e+30
differential_evolution step 1: f(x)= 8.06207e+30
differential_evolution step 2: f(x)= 8.06207e+30
differential_evolution step 3: f(x)= 8.06207e+30
differential_evolution step 4: f(x)= 8.06207e+30
differential_evolution step 5: f(x)= 8.06207e+30
differential_evolution step 6: f(x)= 8.06207e+30
differential_evolution step 7: f(x)= 8.06207e+30
differential_evolution step 8: f(x)= 8.06207e+30
differential_evolution step 9: f(x)= 8.06207e+30
differential_evolution step 10: f(x)= 8.06207e+30
differentia

differential_evolution step 5: f(x)= 8.01343e+30
differential_evolution step 6: f(x)= 8.01343e+30
differential_evolution step 7: f(x)= 8.01343e+30
differential_evolution step 8: f(x)= 8.01343e+30
differential_evolution step 9: f(x)= 8.01343e+30
differential_evolution step 10: f(x)= 8.01343e+30
differential_evolution step 11: f(x)= 8.01343e+30
differential_evolution step 12: f(x)= 8.01343e+30
differential_evolution step 13: f(x)= 8.01343e+30
differential_evolution step 14: f(x)= 8.01343e+30
differential_evolution step 15: f(x)= 8.01343e+30
differential_evolution step 16: f(x)= 8.01343e+30
differential_evolution step 17: f(x)= 8.01343e+30
differential_evolution step 18: f(x)= 8.01343e+30
differential_evolution step 19: f(x)= 8.01343e+30
differential_evolution step 20: f(x)= 8.01343e+30
differential_evolution step 21: f(x)= 8.01343e+30
differential_evolution step 22: f(x)= 8.01343e+30
differential_evolution step 23: f(x)= 8.01343e+30
differential_evolution step 24: f(x)= 8.01343e+30
diffe

differential_evolution step 33: f(x)= 7.95824e+30
differential_evolution step 34: f(x)= 7.95824e+30
differential_evolution step 35: f(x)= 7.95824e+30
differential_evolution step 36: f(x)= 7.95824e+30
differential_evolution step 37: f(x)= 7.95824e+30
differential_evolution step 38: f(x)= 7.95824e+30
differential_evolution step 39: f(x)= 7.95824e+30
differential_evolution step 40: f(x)= 7.95824e+30
differential_evolution step 41: f(x)= 7.95824e+30
differential_evolution step 42: f(x)= 7.95824e+30
differential_evolution step 43: f(x)= 7.95824e+30
differential_evolution step 44: f(x)= 7.95824e+30
differential_evolution step 45: f(x)= 7.95824e+30
differential_evolution step 46: f(x)= 7.95824e+30
differential_evolution step 47: f(x)= 7.95824e+30
differential_evolution step 48: f(x)= 7.95824e+30
differential_evolution step 49: f(x)= 7.95824e+30
differential_evolution step 0: f(x)= 7.95824e+30
differential_evolution step 0: f(x)= 7.95824e+30
differential_evolution step 1: f(x)= 7.95824e+30
dif

differential_evolution step 47: f(x)= 7.86881e+30
differential_evolution step 48: f(x)= 7.86881e+30
differential_evolution step 49: f(x)= 7.86881e+30
differential_evolution step 0: f(x)= 7.86881e+30
differential_evolution step 0: f(x)= 7.86881e+30
differential_evolution step 1: f(x)= 7.86881e+30
differential_evolution step 2: f(x)= 7.86881e+30
differential_evolution step 3: f(x)= 7.86881e+30
differential_evolution step 4: f(x)= 7.86881e+30
differential_evolution step 5: f(x)= 7.86881e+30
differential_evolution step 6: f(x)= 7.86881e+30
differential_evolution step 7: f(x)= 7.86881e+30
differential_evolution step 8: f(x)= 7.86881e+30
differential_evolution step 9: f(x)= 7.86881e+30
differential_evolution step 10: f(x)= 7.86881e+30
differential_evolution step 11: f(x)= 7.86881e+30
differential_evolution step 12: f(x)= 7.86881e+30
differential_evolution step 13: f(x)= 7.86881e+30
differential_evolution step 14: f(x)= 7.86881e+30
differential_evolution step 15: f(x)= 7.86881e+30
differentia

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
result

     fun: 7.489876167779236e+30
     jac: array([ 2.52352112e+29,  2.91604698e+28, -5.45464728e+28, -1.11597735e+29,
        2.87343505e+29,  1.70201726e+29,  2.35002332e+28, -4.16616743e+27,
       -3.95087285e+28, -1.54104172e+28,  6.06987276e+28, -3.16394988e+29,
       -1.54678381e+28,  4.02959577e+27, -1.23830863e+29, -2.17694211e+29,
       -4.10476084e+28, -8.44129944e+28,  3.54433291e+26,  1.46614686e+27,
        1.06138584e+27,  7.96011234e+26,  1.38361840e+27,  1.87653737e+27,
        1.12342293e+27,  1.67680273e+27,  5.62274413e+26,  1.59123434e+27,
        6.56399646e+26,  1.10484558e+27,  1.02794661e+26,  7.15059031e+26,
        8.73022788e+26,  6.08661490e+26,  5.48651025e+26,  1.09561320e+27])
 message: 'Optimization terminated successfully.'
     nit: 41
 success: True
       x: array([-6.78349057e+01, -5.93900163e+00,  2.01267721e+01, -6.16878810e+01,
        1.90071935e+01,  1.80623321e+00, -5.36401787e+01, -3.09467539e-01,
        2.27305658e+01,  1.80770285e+01, -2.